# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [108]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
import numpy as np
from copy import deepcopy
try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [109]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        self.lives=6
        self.alphabet='abcdefghijklmnopqrstuvwxyz'

        full_dictionary_location = "words_250000_train.txt"
        largedict=self.build_dictionary(full_dictionary_location) 
        random.shuffle(largedict)
        split_point = len(largedict) // 2
        self.full_dictionary = largedict#[:split_point]    
        self.guess_dictionary = largedict#[split_point:]       
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        self.current_dictionary = []
        self.previous_guess='!'
        self.n_outcomes=10
        self.find_probabilities()

    
    def find_probabilities(self):
        lhood=np.ones((26,26,self.n_outcomes))
        prior=np.zeros(26)
        lhoodoflength=np.zeros((26,10))
        for word in self.full_dictionary:
            for loc,l1 in enumerate(word):#given l1
                i=alphabet_to_index(l1)
                for j,l2 in enumerate(self.alphabet):#find if l2 occurs
                    if(l2 !=l1): #l1 should not occur again (already guessed)
                        count=word.count(l2)
                        lhood[i,j,min(count,self.n_outcomes-1)]+=1
                    else:
                        lhood[i,j,:]=lhood[i,j,:]*0

                prior[i]+=1
                lhoodoflength[i,min(9,len(word))]+=1

        #laplacesmoothing
        # lhood+=1
        # prior+=1
        self.prior=prior/prior.sum()

        sum_array = np.sum(lhood, axis=2)
        self.likelihood = np.divide(lhood, sum_array[:, :, np.newaxis], out=np.zeros_like(lhood), where=sum_array[:, :, np.newaxis]!=0)
        sum_llhood=np.sum(lhoodoflength,axis=1)
        self.lenllikelihood=np.divide(lhoodoflength, sum_llhood[:, np.newaxis], out=np.zeros_like(lhoodoflength), where=sum_llhood[ :, np.newaxis]!=0)

    def bayesian_update(self,pi,guess,outcome):
        idx=alphabet_to_index(guess)
        probs=self.likelihood[:,idx,outcome]*pi
        return probs/probs.sum()

    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################
        current_alphabet=set(self.alphabet)-set(self.guessed_letters)
        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word[::2].replace("_",".")

        #update prior with guess correct or not
        if self.previous_guess =='!':
            pass
            # length=min(9,len(clean_word))
            # probs=self.lenllikelihood[:,length]*self.pi
            # self.pi=probs/probs.sum()
            # # print(self.pi)
        else:
            # print(self.pi)
            count=clean_word.count(self.previous_guess)
            self.pi=self.bayesian_update(self.pi,self.previous_guess,min(count,self.n_outcomes-1))

            # for loc, letter in enumerate(clean_word):
            #     if self.previous_guess==letter:
            #         self.pi=self.bayesian_update(self.pi,self.previous_guess,min(loc,self.n_outcomes-1))

        
        #for each possible next guess see if it yields maximum entropy
        max_reduction=0
        best_guess='!'
        current_entropy=entropy(self.pi)
        for nextguess in current_alphabet:
            idx=alphabet_to_index(nextguess)
            expected_entropy=[]
            for count in range(self.n_outcomes):
                expected_entropy+=np.dot(self.likelihood[:,idx,count],self.pi)*entropy(self.bayesian_update(self.pi,nextguess,count))
                #np.dot(self.likelihood[:,idx,count],self.pi)
                # for nextguess1 in current_alphabet:
                #     idx1=alphabet_to_index(nextguess)
                #     for count1 in range(self.n_outcomes):
                # expected_entropy+=np.dot(self.likelihood[:,idx1,count1],self.bayesian_update(self.pi,nextguess,count))*np.dot(self.likelihood[:,idx,count],self.pi)*entropy(self.bayesian_update(self.bayesian_update(self.pi,nextguess,count),nextguess1,count1))
            entropy_reduction=current_entropy-expected_entropy
            if entropy_reduction>max_reduction:
                max_reduction=entropy_reduction
                best_guess=nextguess


        if(best_guess=='!' or clean_word.count('.')==1):
            best_guess=self.alphabet[np.argmax(self.pi)]

        self.previous_guess=best_guess
        return best_guess


    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def play_game(self):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = set()
        self.previous_guess='!'
        self.pi=deepcopy(self.prior)
        self.lives=6
        self.current_dictionary = self.full_dictionary
        word = random.choice(self.guess_dictionary )
        blanks = ['_',' '] * len(word)
        while self.lives > 0:
            print(" ".join(blanks))
            guess = self.guess(" ".join(blanks))
        
            if guess in word and guess not in self.guessed_letters:
                print("Correct!")
                for i in range(len(word)):
                    if word[i] == guess:
                        blanks[2*i] = guess
            else:
                print("Incorrect!")
                self.lives -= 1
        
            if '_' not in blanks:
                print("Congratulations! You guessed the word:", word)
                return True
            self.guessed_letters.add(guess)
            
        print("Out of lives! The word was:", word)
        return False

     
def alphabet_to_index(letter):
    """
    Maps English alphabet letters to their corresponding indices.
    
    Parameters:
        letter (str): A single English alphabet letter.
        
    Returns:
        int: The index of the input letter in the English alphabet (0 for 'a', 1 for 'b', ..., 25 for 'z').
             Returns None if the input is not a valid English alphabet letter.
    """
    if len(letter) != 1 or not letter.isalpha() or not letter.isascii():
        return None
    
    # Convert lowercase letter to index
    index = ord(letter.lower()) - ord('a')
    return index

def entropy(pdf):
    """
    Compute the entropy of a probability distribution.

    Args:
    pdf (array-like): Probability distribution function.

    Returns:
    float: Entropy value.
    """
    entropy_val = 0
    for prob in pdf:
        if prob != 0:  # To avoid log(0) which is undefined
            entropy_val -= prob * np.log2(prob)
    return entropy_val



# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [110]:
api = HangmanAPI()

In [116]:
results=[]
for i in range(200):
    results.append(api.play_game())
print(sum(results)/len(results))

_   _   _   _   _   _   _  
Incorrect!
_   _   _   _   _   _   _  
Incorrect!
_   _   _   _   _   _   _  
Incorrect!
_   _   _   _   _   _   _  
Incorrect!
_   _   _   _   _   _   _  
Incorrect!
_   _   _   _   _   _   _  
Correct!
_   _   _   _   _   r   _  
Incorrect!
Out of lives! The word was: pgnttrp
_   _   _   _   _   _  
Correct!
_   e   _   _   e   _  
Correct!
_   e   r   _   e   _  
Incorrect!
_   e   r   _   e   _  
Incorrect!
_   e   r   _   e   _  
Correct!
_   e   r   _   e   s  
Incorrect!
_   e   r   _   e   s  
Incorrect!
_   e   r   _   e   s  
Incorrect!
_   e   r   _   e   s  
Incorrect!
Out of lives! The word was: vermes
_   _   _   _   _   _   _   _   _   _  
Correct!
e   _   _   _   _   _   _   _   e   _  
Correct!
e   _   _   r   _   _   _   _   e   _  
Correct!
e   _   _   r   i   _   _   _   e   _  


C:\Users\nilan\AppData\Local\Temp\ipykernel_24200\2708323363.py:96: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if entropy_reduction>max_reduction:


Correct!
e   _   _   r   i   _   _   _   e   n  
Incorrect!
e   _   _   r   i   _   _   _   e   n  
Incorrect!
e   _   _   r   i   _   _   _   e   n  
Incorrect!
e   _   _   r   i   _   _   _   e   n  
Correct!
e   _   _   r   i   _   _   t   e   n  
Incorrect!
e   _   _   r   i   _   _   t   e   n  
Incorrect!
e   _   _   r   i   _   _   t   e   n  
Incorrect!
Out of lives! The word was: embrighten
_   _   _   _   _   _   _   _   _   _   _   _   _   _   _  
Incorrect!
_   _   _   _   _   _   _   _   _   _   _   _   _   _   _  
Correct!
_   _   _   a   _   _   _   _   _   _   _   _   _   _   _  
Correct!
_   _   _   a   i   _   _   _   _   _   i   _   i   _   _  
Correct!
_   _   _   a   i   _   _   _   _   _   i   _   i   n   _  
Correct!
_   t   _   a   i   _   _   t   _   _   i   _   i   n   _  
Incorrect!
_   t   _   a   i   _   _   t   _   _   i   _   i   n   _  
Correct!
s   t   _   a   i   _   _   t   s   _   i   _   i   n   _  
Incorrect!
s   t   _   a   i   _   _   t   s   _  

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [112]:
# print()
# print(sum_array)
# print(api.full_dictionary[0].count('a'))
# print(np.argmax(prior*0))
# print(entropy(prior*0.01))

# probs=likelihood[:,2,1]*prior
# prior=probs/probs.sum()

print(np.dot(api.likelihood[:,0,0],api.pi))
# print(entropy(prior))


0.41675578692375925


In [113]:
# api.start_game(practice=1,verbose=True)
# [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
# practice_success_rate = total_practice_successes / total_practice_runs
# print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [114]:
# for i in range(1000):
#     print('Playing ', i, ' th game')
#     # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
#     #api.start_game(practice=0,verbose=False)
    
#     # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
#     time.sleep(0.5)

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [115]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

AttributeError: 'HangmanAPI' object has no attribute 'my_status'